In [2]:
key=""

In [3]:
!pip install openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [4]:
import os
import openai
from openai import OpenAI

In [5]:
openai.api_key=key
client=OpenAI(api_key=key)

In [6]:
import pandas as pd
df=pd.read_csv("/content/cti-mcq.tsv",sep='\t')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/cti-mcq.tsv'

In [ ]:
def get_gpt35_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",  # Use a chat model for better results
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150,
        temperature=0.7,
        n=1,
        stop=None
    )
    return response.choices[0].message.content.strip()

def answer_mcq(question, options):
    prompt = f"Question: {question}\n"
    for i, option in enumerate(options):
        prompt += f"{chr(65 + i)}. {option}\n"
    prompt += "Answer:"

    response = get_gpt35_response(prompt)
    # Extract the answer letter (e.g., "A" from "A. To redirect legitimate traffic to a malicious server")
    answer_letter = response.split('.')[0].strip().upper() if '.' in response else response.strip().upper()
    return answer_letter

def process_csv(file_path):
    df = pd.read_csv(file_path, sep='\t')
    df_sample = df.sample(n=250, random_state=1)
    responses = []

    for _, row in df_sample.iterrows():
        question = row['Question']
        options = [row['Option A'], row['Option B'], row['Option C'], row['Option D']]
        response = answer_mcq(question, options)
        responses.append(response)

    df_sample['GPT-4o'] = responses
    return df_sample




In [ ]:
# Example usage
file_path = '/content/cti-mcq.tsv'  # Replace with the path to your CSV file
result_df = process_csv(file_path)
print(result_df[['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'GPT-4o']])

In [ ]:
output_file_path = '/content/GPT4o.csv'  # Replace with the desired output path
result_df.to_csv(output_file_path, index=False)

In [ ]:
gpt=pd.read_csv("/content/GPT4o.csv")
gpt.head()

In [ ]:
def calculate_accuracy(df):
    # Compare model responses to ground truth
    correct = (df['GPT-4o'].str.strip().str.upper() == df['GT'].str.strip().str.upper()).sum()
    total = len(df)
    accuracy = correct / total * 100
    return accuracy

In [ ]:
accuracy = calculate_accuracy(result_df)
print(f"Model Accuracy: {accuracy:.2f}%")